In [10]:
#imports

import numpy as np
import pandas as pd
import itertools
import time
import matplotlib.pyplot as plt
from heapq import nlargest

In [11]:
#helper functions

#np.random to generate random data
def generate_uniform_synthetic_data(N, seed=42, low=1, high=1000):
    np.random.seed(seed)
    locs = [f"L{i}" for i in range(N)]
    demand = np.random.randint(low, high, size=(N, N)).astype(float)
    np.fill_diagonal(demand, 0.0)
    df = pd.DataFrame([(locs[i], locs[j], demand[i,j]) for i in range(N) for j in range(N) if i!=j],
                      columns=["Origin", "Destination", "Demand"])
    return df, demand, locs

#generate heavily biased synthetic data
def generate_biased_synthetic_data(N, seed=42, low=1, high=1000, bias_ratio=0.9, hub_fraction=0.1):

    np.random.seed(seed)
    locs = [f"L{i}" for i in range(N)]

    num_hubs = max(1, int(N * hub_fraction))
    hub_idx = np.random.choice(N, num_hubs, replace=False)
    hubs = [locs[i] for i in hub_idx]

    demand = np.random.randint(low, high, size=(N, N)).astype(float)
    np.fill_diagonal(demand, 0.0)

    bias_multiplier = (bias_ratio / hub_fraction) * 2
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            if (i in hub_idx) or (j in hub_idx):
                demand[i, j] = demand[i, j]*bias_multiplier

    # Normalization
    avg_val = np.mean(demand)
    scale = (high + low) / 2 / avg_val
    demand = demand*scale

    df = pd.DataFrame([(locs[i], locs[j], demand[i, j])
                       for i in range(N) for j in range(N) if i != j],
                      columns=["Origin", "Destination", "Demand"])

    return df, demand, locs

#givena list of chosen vertiports, compute how much demand is captured by it.
def captured_demand(demand_matrix, chosen):
    if len(chosen) < 2:
        return 0
    return demand_matrix[np.ix_(chosen, chosen)].sum()

In [3]:
#algorithms

#greedy algorithm - using numpy to optimize but still have to loop over M
def greedy_station_selection(demand_matrix, M):
    N = demand_matrix.shape[0]
    chosen = []
    remaining = list(range(N))
    conn = demand_matrix.sum(0) + demand_matrix.sum(1)
    first = np.argmax(conn)
    chosen.append(first)
    remaining.remove(first)
    while len(chosen) < M:
        best = None
        best_gain = -1
        for loc in remaining:
            gain = demand_matrix[loc, chosen].sum() + demand_matrix[chosen, loc].sum()
            if gain > best_gain:
                best_gain = gain
                best = loc
        chosen.append(best)
        remaining.remove(best)
    total = captured_demand(demand_matrix, chosen)
    return chosen, total

#just literal brute force, max_time of 60 seconds to abort run
def brute_force_station_selection(demand_matrix, M, max_time=60):
    N = demand_matrix.shape[0]
    start = time.time()
    best = None
    best_val = -1
    for combo in itertools.combinations(range(N), M):
        if (time.time() - start) > max_time:
            print(f"[brute force] TIME LIMIT hit at {round(time.time()-start,2)}s!")
            return (list(best) if best else [], best_val)
        val = captured_demand(demand_matrix, combo)
        if val > best_val:
            best_val = val
            best = combo
    return list(best), best_val


#using branch pruning trick
class BranchAndBoundSolver:
    def __init__(self, demand_matrix, M, time_limit=60, greedy_init=None):
        self.D = demand_matrix
        self.N = demand_matrix.shape[0]
        self.M = M
        self.time_limit = time_limit
        self.best_val = -1
        self.best_set = []
        self.start_time = None
        self.conn = self.D.sum(0) + self.D.sum(1)
        self.visited = 0
        self.pruned = 0

        # If a greedy warm start is provided, use it
        if greedy_init is not None:
            greedy_sel, greedy_val = greedy_init
            self.best_val = greedy_val
            self.best_set = greedy_sel[:]
            print(f"[warm start] Using Greedy init with value={round(greedy_val,2)}")

    def time_up(self):
        return (time.time() - self.start_time) > self.time_limit

    def upper_bound(self, chosen, remaining):
        if len(chosen) <= 1:
            cur = 0
        else:
            cur = captured_demand(self.D, chosen)
        need = self.M - len(chosen)
        if need <= 0:
            return cur
        top = nlargest(need, self.conn[remaining])
        return cur + sum(top)

    def solve(self):
        self.start_time = time.time()
        all_nodes = list(range(self.N))
        all_nodes.sort(key=lambda x: -self.conn[x])
        self.dfs([], all_nodes)
        elapsed = time.time() - self.start_time
        return {"best_set": self.best_set, "best_val": self.best_val,
                "visited": self.visited, "pruned": self.pruned, "time": elapsed}

    def dfs(self, chosen, remaining):
        if self.time_up():
            return
        self.visited += 1
        if len(chosen) == self.M:
            val = captured_demand(self.D, chosen)
            if val > self.best_val:
                self.best_val = val
                self.best_set = chosen[:]
            return
        if not remaining:
            return
        ub = self.upper_bound(chosen, remaining)
        if ub <= self.best_val:
            self.pruned += 1
            return
        for i, node in enumerate(remaining):
            if self.time_up():
                return
            new_chosen = chosen + [node]
            new_remaining = remaining[i+1:]
            if len(new_chosen) + len(new_remaining) < self.M:
                continue
            self.dfs(new_chosen, new_remaining)

In [15]:
#test and compare

# Generate big dataset (N=3000) ONCE
bigN = 1000
print(f"Generating synthetic data for N={bigN} ...")

#different data generation processes
#x, big_demand, big_locs = generate_uniform_synthetic_data(bigN, seed=1234)
#x, big_demand, big_locs = generate_biased_synthetic_data(bigN, seed=1234)
x, big_demand, big_locs = generate_biased_synthetic_data(bigN, seed=1234,  bias_ratio=0.9, hub_fraction=0.01)

pairs = [
    #(10, 3), (10, 5),
    #(50, 3), (50, 5), (50, 10),
    #(100, 3), (100, 5),  (100, 10), (100, 50),
    #(500, 3), (500, 5), (500, 10), (500, 50), (500, 100),
    (1000, 3), (1000, 5), (1000, 10), (1000, 50), (1000, 100), (1000, 300)
    #(3000, 3), (3000, 5), (3000, 10), (3000, 25), (3000, 50), (3000, 100), (3000, 300)
]

results = []

for N, M in pairs:
    print(f"\n=== TEST N={N}, M={M} ===")
    demand = big_demand[:N,:N]
    locs = big_locs[:N]

    #Greedy
    t0 = time.time()
    greedy_sel, greedy_val = greedy_station_selection(demand, M)
    gtime = time.time() - t0
    print(f"[GREEDY] time={round(gtime,3)}s, captured={round(greedy_val,2)}")

    #Brute Force
    t0 = time.time()
    bf_sel, bf_val = brute_force_station_selection(demand, M, max_time=60)
    bftime = time.time() - t0
    bf_forced = bftime > 59.5
    print(f"[BRUTE]  time={round(bftime,3)}s, captured={round(bf_val,2)}{' (FORCED STOP)' if bf_forced else ''}")

    #Branch and Bound (use greedy init)
    solver = BranchAndBoundSolver(demand, M, time_limit=60, greedy_init=(greedy_sel, greedy_val))
    out = solver.solve()
    bbtime = out["time"]
    bb_forced = bbtime > 59.5
    print(f"[B&B]    time={round(bbtime,3)}s, captured={round(out['best_val'],2)}, visited={out['visited']}, pruned={out['pruned']}{' (FORCED STOP)' if bb_forced else ''}")

    results.append({
        "N": N,
        "M": M,
        "Greedy_time": round(gtime,3),
        "Greedy_val": round(greedy_val,2),
        "Brute_time": round(bftime,3),
        "Brute_val": round(bf_val,2),
        "Brute_forced": bf_forced,
        "BB_time": round(bbtime,3),
        "BB_val": round(out["best_val"],2),
        "BB_forced": bb_forced
    })


Generating synthetic data for N=1000 ...

=== TEST N=1000, M=3 ===
[GREEDY] time=0.023s, captured=185544.3
[brute force] TIME LIMIT hit at 60.0s!
[BRUTE]  time=60.0s, captured=194056.96 (FORCED STOP)
[warm start] Using Greedy init with value=185544.3
[B&B]    time=60.0s, captured=218688.47, visited=3441337, pruned=0 (FORCED STOP)

=== TEST N=1000, M=5 ===
[GREEDY] time=0.066s, captured=549172.3
[brute force] TIME LIMIT hit at 60.0s!
[BRUTE]  time=60.0s, captured=385529.46 (FORCED STOP)
[warm start] Using Greedy init with value=549172.3
[B&B]    time=60.0s, captured=614172.14, visited=3463274, pruned=0 (FORCED STOP)

=== TEST N=1000, M=10 ===
[GREEDY] time=0.098s, captured=2060601.47
[brute force] TIME LIMIT hit at 60.0s!
[BRUTE]  time=60.0s, captured=866166.15 (FORCED STOP)
[warm start] Using Greedy init with value=2060601.47
[B&B]    time=60.0s, captured=2160153.31, visited=3114150, pruned=4858 (FORCED STOP)

=== TEST N=1000, M=50 ===
[GREEDY] time=0.755s, captured=22139252.3
[brute f